In [32]:
import pandas as pd
# Python SQL toolkit and Object Relational Mapper
import sqlite3
import sqlalchemy
import json
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session, registry
from sqlalchemy import create_engine, inspect, func, Table, MetaData

In [16]:
# create engine to min_wage.db
engine = create_engine(r'sqlite:///../min_wage.sqlite')

In [17]:
# reflect the tables
Base = automap_base()
Base.prepare(engine, reflect=True)
Pstates = Base.classes.par_states
PYears =Base.classes.par_years
Ppce =Base.classes.par_pce

Wages = Base.classes.wages
Pce =Base.classes.pce
Rpp = Base.classes.rpp


In [20]:
session = Session(engine)
q = session.query(Wages.year,Wages.state_name,Wages.min_wage_eff)
df = pd.DataFrame(q)
session.close()
Base.classes.keys()

['par_pce', 'par_states', 'par_years', 'pce', 'rpp', 'wages']

In [29]:

session = Session(engine)
for row in session.query(Wages.year, Wages.state_name, Wages.min_wage_eff).limit(2).all():
    print (row)
session.close()

(1990, 'Alabama', 6.63)
(1990, 'Alaska', 7.62)


In [ ]:
session = Session(engine)
row= session.query(Wages.min_wage_eff, Wages.state_name, Wages.year).limit(10).all()
rs =json.dumps(dict(row))
rs
session.close()


# Reflect Tables into SQLAlchemy ORM

In [ ]:

# reflect the tables
meta = MetaData()
meta.reflect(engine)

# Base = automap_base(metadata=meta)

# Base.prepare()
inspector = inspect(engine)

print(inspector.get_table_names())

mapper_registry = registry()
# Save references to each table
wages_tbl = Table('wages', meta, autoload=engine)
pce_tbl = Table('pce', meta, autoload=engine)
rpp_tbl = Table('rpp', meta, autoload=engine)
states_tbl = Table('par_states', meta, autoload=engine)
year_tbl = Table('par_years', meta, autoload=engine)

class Wages:
    pass
mapper_registry.map_imperatively(Wages, wages_tbl)


In [3]:
# Create session (link) from Python to the DB
session = Session(bind=engine)

In [5]:
pce_data = session.query(pce_tbl.c.Year,pce_tbl.c.state_name,pce_tbl.c.pce_category,pce_tbl.c.pce_value)
df = pd.DataFrame(pce_data)
df

,0,1,2,3
0,1997,United States,Clothing and footwear,908
1,1997,United States,Gasoline and other energy goods,542
2,1997,United States,Household consumption expenditures (for services),12561
3,1997,United States,Housing and utilities,3704
4,1997,United States,Health care,2901
...,...,...,...,...
5975,2019,Wyoming,Clothing and footwear,878
5976,2019,Wyoming,Gasoline and other energy goods,2352
5977,2019,Wyoming,Household consumption expenditures (for services),29090
5978,2019,Wyoming,Housing and utilities,7877


# Close Session

In [34]:
# Close Session
session.close()